In [1]:
import re

tree_str = "o0(c1;c0),o1(c2;c1)"
operations = tree_str.split(",")
print(operations)
regex = r"(o.*)\((.*)\)"
for operation in operations:
    match = re.match(regex,operation)
    if match:
        node, edges = match.groups()
        print(node, edges)



['o0(c1;c0)', 'o1(c2;c1)']
o0 c1;c0
o1 c2;c1
